In [7]:
from pathlib import Path
import topagnps2cche1d as t2c

# from t2c import Watershed, CrossSection
# from topagnps2cche1d import Watershed, CrossSection

casename = "goodwin_creek_1m"

root_dir = Path(f"../data_inputs/{casename}/")
# root_dir = Path("../data_inputs/goodwin_creek_1m_1_and_3_reaches/TopAGNPS_DataSets/3_Reach")
# root_dir = Path("../data_inputs/topagnps_ohio_files/")

path_to_reaches = root_dir / "AnnAGNPS_Reach_Data_Section.csv"

path_to_cells = root_dir / "AnnAGNPS_Cell_Data_Section.csv"
path_to_reaches_raster = root_dir / "AnnAGNPS_Reach_IDs.asc"

step = 10

In [8]:
watershed = t2c.Watershed()
watershed.import_topagnps_reaches_network(path_to_reaches)
watershed.import_topagnps_cells(path_to_cells)

watershed.read_reaches_geometry_from_topagnps_asc_file(path_to_reaches_raster)

watershed.assign_strahler_number_to_reaches()

Reading Reaches: 100%|██████████| 910/910 [37:26<00:00,  2.47s/it]  


In [9]:
max_strahler = watershed.get_highest_strahler_number()
print(f"The highest Strahler number is {max_strahler}")

The highest Strahler number is 6


In [10]:
strahler = max_strahler - 2 # To get Main and Secondary Channel
watershed.ignore_reaches_with_strahler_leq(strahler)

watershed.update_watershed()

watershed.resample_reaches(id_list="all", step=step)

watershed.assign_cross_section_to_all_points()
watershed.adjust_cross_sections_elevation_with_slope()

In [11]:
df_nodes = watershed.create_cche1d_nodes_df()
df_ch = watershed.create_cche1d_channels_df()
df_lk, df_rc = watershed.create_cche1d_links_and_reaches_df()
df_csec, df_csprf = watershed.create_cche1d_csec_csprf_df()
df_tw = watershed.create_cche1d_twcells_df()
df_inflow_reaches = watershed.get_list_of_inflow_reaches_df()

In [12]:
# casename_strahler_step = f"{casename}_strahler_gt{strahler}_step_{step}m"
casename_strahler_step = f"{casename}_reach_order_{max_strahler - strahler}_step_{step}m"

# step = "native_"
# strahler = 0
# casename = f"ohio_strahler_gt{strahler}_step_{step}m"
watershed.write_cche1d_dat_files(
    casename=casename_strahler_step,
    output_folder=f"./../data_outputs/{casename_strahler_step}",
)

In [13]:
watershed.plot(
    frame_height=700, frame_width=900, line_width=0, by="Strahler_Number", title="Watershed"
)

:Overlay
   .NdOverlay.I  :NdOverlay   [Strahler_Number]
      :Curve   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)
   .NdOverlay.II :NdOverlay   [Strahler_Number]
      :Scatter   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)

: 